In [1]:
from copy import deepcopy
import math
from pathlib import Path

from arcgis.features import FeatureSet, FeatureLayerCollection, FeatureCollection
from arcgis.geometry import SpatialReference
from arcgis.gis import GIS, CreateServiceParameter
import pandas as pd
from tqdm.notebook import tqdm

from reach_tools import Reach

%load_ext autoreload
%autoreload 2

In [2]:
raw_dir_pth = Path.cwd().parent / f"data/raw/american_whitewater/"
url = 'https://adventure.maps.arcgis.com'
arcgis_client_id = "2MonB9785a7oRJEl"

In [3]:
gis = GIS(url, client_id=arcgis_client_id)

gis

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adventure.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=2MonB9785a7oRJEl&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=41k8zPlN2UVkvfbi5W418FNx5a76og&allow_verification=false


Enter code obtained on signing in using SAML:  ········


GIS @ https://adventure.maps.arcgis.com version:2025.1

In [4]:
service_name = 'river_reaches'
overwrite = True

In [5]:
# get the list of all downloaded reach json files
raw_json_lst = sorted(list(raw_dir_pth.glob("aw_*.json")))

In [6]:
# get a dictionary of the maximum lengths for all the string columns plus 10 percent rounded up to the next whole integer
dict_lst = [{k: len(v) if isinstance(v, str) else None for k, v in Reach.from_aw_json(reach_json).attributes.items()} for reach_json in raw_json_lst]

len_dict = pd.DataFrame.from_records(dict_lst).max().dropna().astype(int).apply(lambda val: math.ceil(val * 1.1)).astype(int).to_dict()

In [7]:
# get a list of features as dictionary objects
feature_lst = [Reach.from_aw_json(reach_json).line_feature.as_dict for reach_json in raw_json_lst]

In [8]:
# REF: https://community.esri.com/t5/arcgis-api-for-python-blog/create-a-new-hosted-feature-service-in-arcgis/ba-p/1585791

In [9]:
# creeate a feature set from the list of dictionary objects
fs_prop = FeatureSet.from_dict({"features": feature_lst})

# add length to the feature set field if a length was found above
def add_fld_meta(fld: dict, nullable: bool = True, editable: bool = True) -> dict:

    # get field length if applicable
    fld_len = len_dict.get(fld['name'])
    if fld_len is not None:
        fld['length'] = fld_len
    
    return fld

# add length to the fields and exclude the ObjectID field
fields = [add_fld_meta(fld) for fld in fs_prop.fields if fld['type'] != 'esriFieldTypeOID']

# explicitly add the ObjectID field definition to ensure it is correctly configured
oid_fld = {
    "name" : "OBJECTID",
    "type" : "esriFieldTypeOID",
    "actualType" : "int",
    "alias" : "OBJECTID",
    "sqlType" : "sqlTypeInteger",
    "nullable" : False,
    "editable" : False
}
fields.append(oid_fld)

In [10]:
# start creating the definition
lyr_def = {
    "type" : "Feature Layer",
    "name" : service_name,
    "geometryType" : "esriGeometryPolyline", # esriGeometryPolygon, esriGeometryPolyline, esriGeometryPoint
    "indexes" : [ # required index for Primary Key
        {
            "name" : "PK_IDX",
            "fields" : "OBJECTID",
            "isAscending" : True,
            "isUnique" : True,
            "description" : "clustered, unique, primary key"
        }
    ],
    "objectIdField" : "OBJECTID",
    "uniqueField" : { # a unique field is required
        "name" : "OBJECTID",
        "isSystemMaintained" : True
    },
    "spatialReference" : {
        "wkid" : 4326,
    }, 
}

# add the fields to the definition
lyr_def['fields'] = fields

In [11]:
# if the service already exists, remove it
existing_lst = gis.content.search(service_name)

if len(existing_lst) > 0 and overwrite:
    gis.content.delete_items(existing_lst)

elif len(existing_lst) > 0:
    raise FileExistsError(f'Service "{service_name}" already exists in the ArcGIS Online organization.')

Recycle bin not enabled on this organization. Permanent delete parameter ignored.


In [12]:
# create a new empty service item
new_svc_itm = gis.content.create_service(service_name, service_type='featureService', wkid=4326)

In [13]:
new_svc_itm

<Item title:"river_reaches" type:Feature Layer Collection owner:jmccune_personal>

In [14]:
# create a feature layer collection
flc = FeatureLayerCollection.fromitem(new_svc_itm)

# add the layer definition to the item
flc.manager.add_to_definition({'layers': [lyr_def]})

{'success': True, 'layers': [{'name': 'river_reaches', 'id': 0}]}

In [15]:
# get the newly created layer to work with
lyr = new_svc_itm.layers[0]

In [16]:
chunk_size = 200
upload_chunks = [feature_lst[idx: idx + chunk_size] for idx in range(0, len(feature_lst), chunk_size)]

In [17]:
for chnk in tqdm(upload_chunks):
    lyr.edit_features(adds=chnk)

  0%|          | 0/26 [00:00<?, ?it/s]